This example shows how to use pytraj (wrapper of cpptraj) and pysander (wrapper of sander) to extract useful (energy) information (energy) for post-processing MD data.

Aim: load traj and calculate solvation energy for 10 frames of Trp-cage MD simulation using igb=8 model

In [1]:
from __future__ import print_function

In [2]:
# require: pytraj, pysander and chemistry packages (pysander and `chemistry` are in AmberTools 15)

In [3]:
# load `pytraj.io` method to load trajectory
from pytraj import io as mdio

In [4]:
# load sander from pysander
import sander

In [5]:
# load AmberParm class
from chemistry.amber.readparm import AmberParm

In [6]:
traj_fn = "../tests/data/md1_prod.Tc5b.x"
top_fn = "../tests/data/Tc5b.top"

In [7]:
# load traj
traj = mdio.load(traj_fn, top_fn)
print (traj)
print (traj.top)

<pytraj.Trajectory with 10 frames: <Topology with 1 mols, 20 residues, 304 atoms, 310 bonds, non-PBC>>
           
<Topology with 1 mols, 20 residues, 304 atoms, 310 bonds, non-PBC>


In [8]:
# currently `chemistry` and pytraj packges use different Topology objects so we need to load parm file again
#
parm = AmberParm(top_fn)

In [9]:
print (parm)

../tests/data/Tc5b.top


In [10]:
# create input for igb=8 model
inp = sander.gas_input(8)

In [11]:
# initialize parm
parm.load_coordinates(traj[0].coords)

In [12]:
# it's time to do calcuation
with sander.setup(parm, parm.coords, None, inp):
    # iterate all frames
    # use traj(0, 8, 2) to read the traj from 0-th frame to 8-th frame but skiping every 2 frames
    for frame in traj:
        sander.set_positions(frame.coords)
        ene, forces = sander.energy_forces()
        # atomic force (x, y, z)
        #print (len(frc))
        print ("Solvation energy = ", ene.gb)

Solvation energy =  -466.10420332687943
Solvation energy =  -455.76754993695613
Solvation energy =  -455.5976464241175
Solvation energy =  -478.5433764963464
Solvation energy =  -448.8931420795517
Solvation energy =  -404.6002687139827
Solvation energy =  -450.3030628951594
Solvation energy =  -414.40620305552153
Solvation energy =  -443.75073691232376
Solvation energy =  -429.7327396323983


## shorten

In [13]:
import pytraj.common_actions as pyca

# calculate energy with default value (igb=8)
energies = pyca.energy_decomposition (traj, dtype='dataset')
energies

   amd_boost       angle  angle_ub       bond  cmap  constraint  ct  \
0          0  162.344881         0  69.750852     0           0   0   
1          0  171.913530         0  67.323230     0           0   0   
2          0  160.140173         0  54.356569     0           0   0   
3          0  175.304762         0  58.912320     0           0   0   
4          0  179.411735         0  80.720256     0           0   0   
5          0  170.047861         0  66.271355     0           0   0   
6          0  178.860154         0  81.916833     0           0   0   
7          0  157.791251         0  77.624779     0           0   0   
8          0  176.649974         0  74.490781     0           0   0   
9          0  160.988947         0  76.494733     0           0   0   

     dihedral  disp  dvdl    ...      les  noe  pb  polar  rism  scf  surf  \
0  233.003209     0     0    ...        0    0   0      0     0    0     0   
1  244.214391     0     0    ...        0    0   0      0     

In [14]:
# full options
help(pyca.energy_decomposition)

Help on function get_pysander_energies in module pytraj.externals.get_pysander_energies:

get_pysander_energies(traj=None, parm=None, igb=8, input_options=None, qmmm_options=None, mode=None, top=None, dtype='dict')
    "
    Parameters
    ---------
    traj : {Traj-like object, frame, list of trajs, list of frames} from pytraj
        if `traj` does not hold Topology information, `top` must be provided
    parm : {str, Topology object from ParmEd}, default=None, optional
    igb : GB model, default=8 (GB-Neck2)
        Note: this `igb` input will be ignored if `input_options` is not None
    input_options : InputOptions object from `sander`, default=None, optional
        if `input_options` is None, use `gas_input` with `igb = 8`
        If `input_options` is not None, use this
    qmmm_options : InputOptions object from `sander` for QMMM, optional
    mode : str, default=None, optional
        if mode='minimal', get only 'bond', 'angle', 'dihedral' and 'total' energies
    top : {Top

In [15]:
# just care about some 'minimal' output?
energies = pyca.energy_decomposition (traj, dtype='dataset', mode='minimal')
energies # require pandas to have pretty print (but don't need pandas for getting raw data at all)

        angle       bond    dihedral         tot
0  162.344881  69.750852  233.003209 -386.855858
1  171.913530  67.323230  244.214391 -382.909194
2  160.140173  54.356569  255.441798 -399.192386
3  175.304762  58.912320  236.116012 -404.064231
4  179.411735  80.720256  248.271116 -370.239771
5  170.047861  66.271355  241.952192 -390.839152
6  178.860154  81.916833  244.148374 -375.029568
7  157.791251  77.624779  247.066556 -394.649356
8  176.649974  74.490781  245.516936 -392.685578
9  160.988947  76.494733  258.293820 -384.199356

In [16]:
print (energies.values) # to ndarray

[[ 162.344881    171.91353045  160.1401726   175.30476242  179.41173458
   170.04786071  178.86015408  157.7912515   176.64997445  160.98894657]
 [-386.85585763 -382.90919447 -399.1923861  -404.06423108 -370.23977142
  -390.83915224 -375.02956799 -394.64935554 -392.68557807 -384.19935564]
 [ 233.00320908  244.21439112  255.44179787  236.11601154  248.27111562
   241.95219203  244.14837443  247.06655637  245.51693608  258.29382019]
 [  69.75085151   67.32323037   54.35656918   58.91231967   80.72025559
    66.27135516   81.91683307   77.62477894   74.4907812    76.49473283]]


In [17]:
# full output with return type as a dictionary
energies = pyca.energy_decomposition (traj, dtype='dict')
energies['gb'] # get GB energies

array('d', [-466.10420332687943, -455.76754993695613, -455.5976464241175, -478.5433764963464, -448.8931420795517, -404.6002687139827, -450.3030628951594, -414.40620305552153, -443.75073691232376, -429.7327396323983])

In [18]:
# all supported keys
print (energies.keys())

dict_keys(['elec', 'scf', 'angle_ub', 'rism', 'dvdl', 'imp', 'gb', 'cmap', 'surf', 'pb', 'polar', 'disp', 'vdw', 'emap', 'les', 'amd_boost', 'bond', 'ct', 'hbond', 'noe', 'angle', 'elec_14', 'constraint', 'vdw_14', 'tot', 'dihedral'])
